# SCRIPT DEMO

###### 

###### 

----------------------------

## Test de la collection

###### 

In [3]:
import psycopg2
import os

try:
    conn = psycopg2.connect(
            user = "postgres",
            password = os.environ.get("pg_psw"),
            host = "localhost",
            port = "5432",
            database = "collection5"
    )
    cur = conn.cursor()

    liste_utilisateurs = [('mayel', 'apprenant', 'mayel@simplon.com'), ('joris', 'super apprenant', 'joris@simplon.com'), ('pierre', 'formateur', 'joris@simplon.com')]
    requete_sql1 = """INSERT INTO utilisateur (nom, surnom, email) VALUES (%s,%s,%s)"""
    
    for utilisateur in liste_utilisateurs:

        cur.execute(requete_sql1, utilisateur)

    conn.commit()
    print('insertion terminée')
except psycopg2.Error as e:
    
    print(e)

finally:
        
    # ferme le curseur et devient inutilisable

    cur.close()

    # ferme la connexion à database

    conn.close()

insertion terminée


In [4]:
import psycopg2
import os

try:
    conn = psycopg2.connect(
            user = "postgres",
            password = os.environ.get("pg_psw"),
            host = "localhost",
            port = "5432",
            database = "collection5"
    )
    cur = conn.cursor()

    liste_collections = [('mayelcollec', 'amateur', '1'), ('joriscollec', 'super spéculation', '2'), ('pierrecollec', 'découverte', '3')]
    requete_sql2 = """INSERT INTO collection (nom, description, code_utilisateur) VALUES (%s,%s,%s)"""
    
    for collection in liste_collections:

        cur.execute(requete_sql2, collection)

    conn.commit()
    print('insertion terminée')
    
except psycopg2.Error as e:
    
    print(e)

finally:
        
    # ferme le curseur et devient inutilisable

    cur.close()

    # ferme la connexion à database

    conn.close()

ERREUR:  la valeur d'une clé dupliquée rompt la contrainte unique « collection_code_utilisateur_key »
DETAIL:  La clé « (code_utilisateur)=(1) » existe déjà.



In [5]:
import psycopg2
import os
import random as r


try:
    # fonction d'insertion dans la table compose
    conn = psycopg2.connect(
            user = "postgres",
            password = os.environ.get("pg_psw"),
            host = "localhost",
            port = "5432",
            database = "collection5"
    )
    cur = conn.cursor()
    for i in range(1,1000):
        code_produit = r.randint(1,8400)
        code_collection = r.randint(1,3)

        cur.execute(f"""INSERT INTO compose (code_produit, code_collection)
                        VALUES(%s,%s)""", (code_produit, code_collection))

    cur.close()
    conn.commit()
    print('insertion terminée')

except psycopg2.Error as e:
    
    print(e)

finally:
        
    # ferme le curseur et devient inutilisable

    cur.close()

    # ferme la connexion à database

    conn.close()
    

ERREUR:  la valeur d'une clé dupliquée rompt la contrainte unique « compose_pkey »
DETAIL:  La clé « (code_produit, code_collection)=(6394, 1) » existe déjà.



In [6]:
import pandas as pd

conn = psycopg2.connect(
            user = "postgres",
            password = os.environ.get("pg_psw"),
            host = "localhost",
            port = "5432",
            database = "collection6"
    )

df2 = pd.read_sql("""SELECT count(produit.code_produit)
FROM produit;""", conn)

df2

,count
0,8572


###### 

-------------------------

# DEMO INTEGRATION NOUVEUX JEUX DE DONNEES

###### 

In [7]:
from collections import defaultdict
import pandas as pd
import re
import json

origines = []
degres = []
volumes = []
noms = []
producteurs = []
marques = []
liste_json = []

with open('rumy.json', 'r') as f:
    data = json.load(f)

for rum in data:
    for k, v in rum.items():
        v = v.replace(' cl', 'cl')
        liste_json.append(v)
        
for element in liste_json:
    
    # Origines
    pays = element.split()[-1]
    origines.append(pays)
    if pays:
        element = element.replace(pays, "")
        
    # Producteurs  
    marque_pattern = re.compile(r"[a-zA-Zèé]+[&? [a-zA-Zèé0-9']+ Rhum|[a-zA-Zèé']+[ ]? Rhum|[a-zA-Zèé']+ [Liqueur|Punch|Coffret]+")
    marque = marque_pattern.findall(element)
#     marque = marque.replace("Rhum", "")
    if marque:
        marques.append(marque[0])
    else:
        marques.append("XXXXXXXX")

    # Degres    
    degre_pattern = re.compile(r'\d\d°|\d\d,\d\d?°|\d\d\.\d\d?°')
    degre = degre_pattern.findall(element)
    if degre:
        element = element.replace(degre[0], "")
        degres.append(degre[0])
    else:
        degres.append("")
        
    # Volumes
    volume_pattern = re.compile(r'\d?\dcl|\dL')
    volume = volume_pattern.findall(element)
    if volume:
        element = element.replace(volume[0], "")
        volumes.append(volume[0])
    else:
        volumes.append('')
        
    # Noms rhums    
    noms.append(element) 

# nettoyage Producteurs   
for producteur in marques:
    producteur = re.sub(' Rhum| Liqueur| Punch| Coffret', '', producteur)
    producteurs.append(producteur)
    


In [8]:
print(len(producteurs))

105


In [9]:
import pandas as pd

df1 = pd.DataFrame()
df1['nom'] = noms
df1['producteur'] = producteurs
df1['origine'] = origines
df1['volume'] = volumes
df1['degre'] = degres
df1.shape

(105, 5)

In [10]:
df1.to_csv('rumy_json_1.csv', index = False, header=True)

In [11]:
df1 = pd.read_csv('rumy_json_1.csv')

# df1['producteur'] = df1['producteur'].replace(['Maison L', 'Maison La Mauny'], 'La Mauny') 

# df1.loc[df1['producteur'] == 'Maison La Mauny', 'producteur'] = 'La Mauny'
df1['origine'] = df1['origine'].replace(('70cl', '90g', '58°'), 'Guyana') 
df1.head(241)

,nom,producteur,origine,volume,degre
0,Eminente Rhum Vieux 7 ans Reserva,Eminente,Cuba,70cl,"41,3°"
1,La Progresiva Rhum Vieux,La Progresiva,Cuba,NaN,41°
2,Black Tears Rhum Épice,Black Tears,Cuba,NaN,40°
3,Conde de Liqueur Elixir,XXXXXXXX,Cuba,70cl,32°
4,Legendario Rhum Ambré Ron Dorado,Legendario,Cuba,70cl,38°
...,...,...,...,...,...
100,QuoRhum Rhum Vieux 12 Solera étui République,QuoRhum,Dominicaine,70cl,40°
101,QuoRhum Rhum Vieux 23 Solera étui République,QuoRhum,Dominicaine,70cl,40°
102,Ron Esclavo Rhum Vieux 12 République,Ron Esclavo,Dominicaine,70cl,40°
103,Ron Esclavo Rhum Vieux 15 République,Ron Esclavo,Dominicaine,70cl,38°


In [64]:
df1.to_csv('rumy_json_2.csv', index = False, header=True)

In [77]:
df2 = pd.read_csv('rumy_json_2.csv')
df2.head

<bound method NDFrame.head of                                                  nom     producteur  \
0              Eminente Rhum Vieux  7 ans Reserva          Eminente   
1                        La Progresiva Rhum Vieux     La Progresiva   
2                           Black Tears Rhum Épice      Black Tears   
3                        Conde de  Liqueur Elixir          XXXXXXXX   
4                Legendario Rhum Ambré Ron Dorado        Legendario   
..                                               ...            ...   
100  QuoRhum Rhum Vieux 12 Solera étui   République         QuoRhum   
101  QuoRhum Rhum Vieux 23 Solera étui   République         QuoRhum   
102          Ron Esclavo Rhum Vieux 12   République     Ron Esclavo   
103          Ron Esclavo Rhum Vieux 15   République     Ron Esclavo   
104          Ron Esclavo Rhum Vieux XO   République     Ron Esclavo   

         origine volume  degre  
0           Cuba   70cl  41,3°  
1           Cuba    NaN    41°  
2           Cuba  

In [78]:
dico = {}
liste_car = ['XXXXXXXX', 'XM','é','è', 'É']
liste_remp = ['Conde', 'Xm','e', 'e', 'e']
# création du dictionnaire avec une compréhension de dictionnaire
dico = {k: v for k, v in zip(liste_car, liste_remp)}

In [79]:
dico2 = {}
liste_car = ['cl', '°', ',']
liste_remp = ['','', '.']
# création du dictionnaire avec une compréhension de dictionnaire
dico2 = {k: v for k, v in zip(liste_car, liste_remp)}

In [80]:
# il faut que regex=True 
df2.replace({'producteur': dico}, regex=True,inplace=True)
df2.replace({'origine': dico}, regex=True,inplace=True)
df2.replace({'volume':dico2}, regex=True, inplace=True)
df2.replace({'degre':dico2}, regex=True, inplace=True)

In [81]:
df2.to_csv('rumy_json_3.csv', index = False, header=True)

In [82]:
df3 = pd.read_csv('rumy_json_3.csv')
df3.head()

,nom,producteur,origine,volume,degre
0,Eminente Rhum Vieux 7 ans Reserva,Eminente,Cuba,70.0,41.3
1,La Progresiva Rhum Vieux,La Progresiva,Cuba,NaN,41.0
2,Black Tears Rhum Épice,Black Tears,Cuba,NaN,40.0
3,Conde de Liqueur Elixir,Conde,Cuba,70.0,32.0
4,Legendario Rhum Ambré Ron Dorado,Legendario,Cuba,70.0,38.0


In [83]:
df3['volume'].astype(float)
df3['degre'].astype(float)

0      41.3
1      41.0
2      40.0
3      32.0
4      38.0
       ... 
100    40.0
101    40.0
102    40.0
103    38.0
104    42.0
Name: degre, Length: 105, dtype: float64

In [84]:
df3.loc[df3.nom.str.contains('Blanc'), 'type'] = 'Blanc'
df3.loc[df3.nom.str.contains('Vieux'), 'type'] = 'Vieux'
df3.loc[df3.nom.str.contains('Ambre'), 'type'] = 'Ambre'
df3.loc[df3.nom.str.contains('Liqueur |Creme | Punch | Epice | Arrange | arrange |Bumbu'), 'type'] = 'Spiced'
df3.loc[df3.nom.str.contains('Coffret |coffret'), 'type'] = 'Coffret'
df3

,nom,producteur,origine,volume,degre,type
0,Eminente Rhum Vieux 7 ans Reserva,Eminente,Cuba,70.0,41.3,Vieux
1,La Progresiva Rhum Vieux,La Progresiva,Cuba,NaN,41.0,Vieux
2,Black Tears Rhum Épice,Black Tears,Cuba,NaN,40.0,NaN
3,Conde de Liqueur Elixir,Conde,Cuba,70.0,32.0,Spiced
4,Legendario Rhum Ambré Ron Dorado,Legendario,Cuba,70.0,38.0,NaN
...,...,...,...,...,...,...
100,QuoRhum Rhum Vieux 12 Solera étui République,QuoRhum,Dominicaine,70.0,40.0,Vieux
101,QuoRhum Rhum Vieux 23 Solera étui République,QuoRhum,Dominicaine,70.0,40.0,Vieux
102,Ron Esclavo Rhum Vieux 12 République,Ron Esclavo,Dominicaine,70.0,40.0,Vieux
103,Ron Esclavo Rhum Vieux 15 République,Ron Esclavo,Dominicaine,70.0,38.0,Vieux


In [85]:
df3['producteur'] =df3['producteur'].str.capitalize()

In [86]:
df3.to_csv('rumy_final.csv', index = False, header=True)